<a href="https://colab.research.google.com/github/david12p/Datathon/blob/main/Datathon_David_de_Paz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql.connector

import xlrd
import mysql.connector
import pandas as pd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 37.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp310-cp310-linux_x86_64.whl size=247953 sha256=2580af30e3366d29cde735ae92b644ecb7c3dd8a7694d88be88b8646e4c72f7e
  Stored in directory: /root/.cache/pip/wheels/76/48/9b/da67ff1a18fe8e9d428f9b1a177716d4a7d363d2bbe83bf6cf
Successfully built mysql.connector


In [ ]:
#Celda oculta para github con las credenciales para la conexión a la bbdd

In [ ]:
nombre_base_datos = "datathon"
nombre_tabla_ventas = "ventas"
nombre_tabla_calendario = "calendario"
nombre_tabla_pedidos = "pedidos"

cursor = conexion.cursor()

cursor.execute(f"USE {nombre_base_datos}")

In [ ]:
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_ventas}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_ventas} ("
               "FAMILIA VARCHAR(255),"
               "Tipo VARCHAR(255),"
               "FechaVenta DATE,"
               "HoraVenta INT,"
               "Articulo VARCHAR(255),"
               "Cantidad FLOAT,"
               "Precio FLOAT,"
               "Importe FLOAT"
               ")")

In [ ]:
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_calendario}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_calendario} ("
               "Fecha DATE,"
               "Festivo VARCHAR(255)"
               ")")

In [ ]:
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_pedidos}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_pedidos} ("
               "Tipo VARCHAR(255),"
               "Fecha DATE,"
               "Articulo VARCHAR(255),"
               "Cantidad FLOAT,"
               "Precio FLOAT,"
               "Importe FLOAT"
               ")")

In [ ]:
# Cargar el archivo Excel en Colab
nombre_archivo_ventas = "ArticulosPanaderia.xlsx"

# Abrir el archivo Excel
df = pd.read_excel(nombre_archivo_ventas)

In [ ]:
# Preparar los datos para la inserción en lotes
registros = []
for _, row in df.iterrows():
    familia = row['FAMILIA']
    tipo = row['Tipo']
    fecha_venta = row['FechaVenta'].strftime('%Y-%m-%d')
    hora_venta = row['HoraVenta']
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append((familia, tipo, fecha_venta, hora_venta, articulo, cantidad, precio, importe))

# Crear la consulta SQL para insertar los datos en la tabla
consulta = f"INSERT INTO {nombre_tabla_ventas} (FAMILIA, Tipo, FechaVenta, HoraVenta, Articulo, Cantidad, Precio, Importe) " \
           "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [ ]:
# Cargar el archivo Excel en Colab
nombre_archivo_calendario = "Calendario.xlsx"

# Abrir el archivo Excel
df = pd.read_excel(nombre_archivo_calendario)

# Preparar los datos para la inserción en lotes
registros = []
for _, row in df.iterrows():
    fecha = row['Fecha'].strftime('%Y-%m-%d')
    festivo = row['Festivo']
    registros.append((fecha,festivo))

# Crear la consulta SQL para insertar los datos en la tabla
consulta = f"INSERT INTO {nombre_tabla_calendario} (Fecha, Festivo) " \
           "VALUES (%s, %s)"

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [ ]:
# Cargar el archivo Excel en Colab
nombre_archivo_pedidos = "CantidadPedida.xlsx"

# Abrir el archivo Excel
df = pd.read_excel(nombre_archivo_pedidos)

# Preparar los datos para la inserción en lotes
registros = []
for _, row in df.iterrows():
    tipo = row['Tipo']
    fecha_venta = row['Fecha'].strftime('%Y-%m-%d')
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append(( tipo, fecha_venta, articulo, cantidad, precio, importe))

# Crear la consulta SQL para insertar los datos en la tabla
consulta = f"INSERT INTO {nombre_tabla_pedidos} (Tipo, Fecha, Articulo, Cantidad, Precio, Importe) " \
           "VALUES (%s, %s, %s, %s, %s, %s)"

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

Por aqui voy

In [ ]:
# Nota: las consultas siguientes están preparadas para lanzarse con los datos cargados en la bbdd data y con los nombres iniciales, si se lanzan con usuario1 debe adaptarse

consulta_proceso="""


/*Ordenamos los productos por familia*/
drop table if exists datathon.articulos_top;
create table datathon.articulos_top as (

	select
	Articulo,
	FAMILIA,
	sum(importe) as importe_total,
	ROW_NUMBER () OVER(partition by FAMILIA order by sum(importe) desc) as orden
	from datathon.ventas
	where FechaVenta >= '2021-05-01'
	group by 1,2

);


/*Creamos un calendario general del 1/1/2017 al 31/12/2023*/
drop table if exists datathon.calendario_dias;
create table datathon.calendario_dias as (

with recursive cte_calendario as (
	select date('2017-01-01') as calendar_date -- fecha de inicio
	union all
	select date_add(calendar_date, interval 1 day) as calendar_date from cte_calendario
	where date_add(calendar_date, interval 1 day) <= date('2023-12-31') -- condicion de fin, fecha final
)

select
calendar_date as fecha,
year(calendar_date) as fx_anno,
month(calendar_date) as fx_mes,
day(calendar_date) as fx_day,
date_format(calendar_date, '%Y%m') as fx_anno_mes,
date_format(calendar_Date,'%x-%v') as semana -- formato
from cte_calendario

);

/*Añadimos al calendario los festivos*/
drop table if exists datathon.calendario_completo;
create table datathon.calendario_completo as (
select
base.*,
festivos.festivo
from datathon.calendario_dias base
left join
	(select
	a.*,
	row_number() over(partition by a.fecha order by a.festivo) as orden
	from datathon.calendario a
	) festivos
on base.fecha=festivos.fecha
and festivos.orden=1
);

/*Agregamos las ventas por día (quitamos el detalle por hora)*/

/*Añadimos además el orden del producto y si está en el rango de fecha de estudio*/
drop table if exists datathon.ventas_diarias;
create table datathon.ventas_diarias as (
select
base.familia,
base.tipo,
base.fechaVenta,
calendario.festivo,
base.articulo,
sum(base.precio*base.cantidad)/sum(base.cantidad) as precio, /* Se calcula como el precio promedio del día, ya que hay cambios de precio a lo largo del día*/
articulos.orden as orden_articulo_familia,
case when base.fechaVenta >=date('2021-05-01') then 'S' else 'N' end as in_fecha_estudio,
sum(base.cantidad) as cantidad,
sum(base.importe) as importe

from datathon.ventas base

inner join datathon.calendario_completo calendario
on base.FechaVenta=calendario.fecha

inner join datathon.articulos_top articulos
on base.familia=articulos.familia
and base.articulo=articulos.articulo


group by 1,2,3,4,5,7,8);


/*Creamos una vista final con el filtrado para el estudio*/
drop view if exists datathon.ventas_diarias_estudio_completo;
create view datathon.ventas_diarias_estudio_completo as
(select *
from datathon.ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and orden_articulo_familia<=5);


/*Creamos una vista extra con el filtrado para el estudio que no tenga días de mayo 23 (que habrá que predecir)*/
drop view if exists datathon.ventas_diarias_estudio;
create view datathon.ventas_diarias_estudio as
(select *
from datathon.ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and fechaventa < date('2023-05-01')
and orden_articulo_familia<=5);
"""

In [ ]:
# Separar las consultas en una lista
consultas_lista = consulta_proceso.split(";")

# Eliminar espacios en blanco y elementos vacíos
consultas_lista = [consulta.strip() for consulta in consultas_lista if consulta.strip()]

# Ejecutar las consultas
for consulta in consultas_lista:
    cursor.execute(consulta)

# Confirmar los cambios
conexion.commit()

In [ ]:
# cursor.execute(f"grant select on {nombre_base_datos}.ventas_diarias to usuario1")
# cursor.execute(f"grant select on {nombre_base_datos}.ventas_diarias_estudio to usuario1")

In [ ]:
# Consultas adicionales por problema de permisos del usuario usuario1 en la base de datos data

In [ ]:
cursor.execute(f"drop table if exists classicmodels.ventas_diarias_estudio;")

In [ ]:
cursor.execute(f"CREATE TABLE classicmodels.ventas_diarias_estudio as SELECT * FROM {nombre_base_datos}.ventas_diarias_estudio")